In [10]:
import dlt
import pandas as pd
import mysql.connector
import boto3
import os
from dotenv import load_dotenv

load_dotenv()

True

In [11]:

AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
AWS_REGION = os.environ.get('AWS_REGION')
AWS_BUCKET_NAME = os.environ.get('AWS_BUCKET_NAME')
DB_HOST = os.environ.get('DB_HOST')
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASS')
DB_NAME = os.environ.get('DB_NAME')
DB_PORT = os.environ.get('DB_PORT')

In [3]:
DATABASE_CONFIG = {
    'user': DB_USER,
    'password': DB_PASSWORD,
    'host': DB_HOST,
    'database': DB_NAME,
    'port': DB_PORT
}

In [14]:
pipeline = dlt.pipeline(
    pipeline_name="mysql_to_s3",
    destination="filesystem",
    dataset_name="mysql_data"
)


In [4]:
conn = mysql.connector.connect(**DATABASE_CONFIG)
cursor = conn.cursor()

In [5]:
cursor.execute("SHOW TABLES")
tables = [table[0] for table in cursor.fetchall()]

In [7]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

In [12]:
print(f"Uploading to bucket: {AWS_BUCKET_NAME}")

Uploading to bucket: dlt-test-1


In [13]:
for table in tables:
    query = f"SELECT * FROM `{table}`"
    df = pd.read_sql(query, conn)

    # Save as Parquet
    parquet_filename = f"{table}.parquet"
    df.to_parquet(parquet_filename, engine="pyarrow")

    # Upload to S3
    s3_client.upload_file(parquet_filename, AWS_BUCKET_NAME, parquet_filename)

    # Remove local file after upload
    os.remove(parquet_filename)

print("Data extraction and upload completed successfully!")

# Close connection
cursor.close()
conn.close()

/tmp/ipykernel_90417/3642827200.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data extraction and upload completed successfully!
